## Initial Setup

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

## Import & Prepare Data

In [4]:
ridership = pd.read_csv("Data/ridership.csv")
ridership.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count,total_precip,avg_temp
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4,3.54,34.6
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4,3.54,34.6
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4,3.54,34.6
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4,3.54,34.6
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4,3.54,34.6


In [5]:
stop_usage = pd.read_csv("Data/stop_usage.csv")
stop_usage.head()

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,...,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs,total_precip,avg_temp
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00,5.34,69.0
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00,5.34,69.0
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05,5.34,69.0
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00,5.34,69.0
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00,3.69,35.2


In [6]:
len(stop_usage)

107611

In [7]:
stop_usage = stop_usage[~stop_usage['avg_ons'].isna()]

In [8]:
len(stop_usage)

107405

### Sort temporal data

In [10]:
def prep_data(df, time_var, target_var, drop_vars):
    df_sorted = df.sort_values(time_var)

    ## establish feature and target dataframes
    X = df_sorted.drop(drop_vars, axis = 1)
    y = df_sorted[target_var]

    ## drop sparse columns
    sparse_columns = list(X.columns[X.nunique() / len(X) * 100 < 0.01])
    X = X.drop(sparse_columns, axis = 1)
    
    return [X, y]

### Establish feature and 

In [12]:
# Establish X and y
datasets = {'ridership': [], 'stop usage': []}
datasets['ridership'] = prep_data(ridership, time_var = 'month_start', target_var = 'avg_riders', drop_vars = ['avg_riders', 'route_full_name'])
datasets['stop usage'] = prep_data(stop_usage, time_var = 'datekey', target_var = 'total_ons', drop_vars = ['stop_name', 'total_ons'])

### Preprocessing

In [14]:
def preprocess_steps(X):
    # Identify column types
    int_cols = X.select_dtypes(include='int').columns.tolist()
    float_cols = X.select_dtypes(include='float').columns.tolist()
    cat_cols = X.select_dtypes(include='object').columns.tolist()

    # Preprocessing
    numeric_preprocessor = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    categorical_preprocessor = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer([
        ('num', numeric_preprocessor, int_cols + float_cols),
        ('cat', categorical_preprocessor, cat_cols)
    ])
    
    # Define XGBoost and LightGBM pipelines
    models = {
        "Baseline (Mean)": DummyRegressor(strategy='mean'),
        "Linear Regression": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', LinearRegression())
        ]),
        "Random Forest": Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', RandomForestRegressor(n_estimators=25, random_state=47, n_jobs=-1))
        ]),
        'XGBoost': Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
        ]),
        'LightGBM': Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
        ])
    }
    return models

## Fit Models

In [16]:
tscv = TimeSeriesSplit(n_splits = 5)
for key,value in datasets.items():
    print(f"Predictions for {key}:")
    X = value[0]
    y = value[1]
    models = preprocess_steps(X)
    for name, model in models.items():
        scores = cross_val_score(model, X, y, cv = tscv, scoring = 'neg_root_mean_squared_error', n_jobs = -1)
        rmse_scores = -scores
        print(f"{name:<20} | Mean RMSE: {rmse_scores.mean():>10,.3f} | Std: {rmse_scores.std():.3f}")
    print("")

Predictions for ridership:
Baseline (Mean)      | Mean RMSE:  1,292.428 | Std: 284.978
Linear Regression    | Mean RMSE:    900.410 | Std: 283.068
Random Forest        | Mean RMSE:    588.167 | Std: 362.441
XGBoost              | Mean RMSE:    634.341 | Std: 270.685
LightGBM             | Mean RMSE:    607.020 | Std: 257.932

Predictions for stop usage:
Baseline (Mean)      | Mean RMSE:    324.453 | Std: 202.314
Linear Regression    | Mean RMSE:    121.240 | Std: 48.062
Random Forest        | Mean RMSE:    102.847 | Std: 127.301
XGBoost              | Mean RMSE:    134.576 | Std: 116.734
LightGBM             | Mean RMSE:    120.884 | Std: 141.135



## Consider PCA

In [171]:
## Note - the below code is not currently working
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

X = datasets['ridership'][0]
X_scaled = preprocessor.fit_transform(X)

y = datasets['ridership'][1]

xy = np.column_stack((X_scaled, y))

pca_data = pca.fit_transform(xy)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 22317

In [ ]:
np.sum(pca.explained_variance_ratio_)

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')
plt.show()

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance');
plt.show()